# Intro

This notebook reads silver tables and performs several transformations to get a company and contact table built, which Census will use.


## Change History

<style>
  table {margin-left: 0 !important;}
</style>

| Date    | Author | Description |
| :-------- | :------- | :------- | 
|2025-07-01 | Mclain R |  Created Date|

# Code

## Imports

###### pyspark.sql.functions
- **F**: A common alias for importing PySpark SQL functions, allowing access to various built-in functions (e.g., F.lit(), F.when(), etc.) for DataFrame transformations.
- **col**: A function used to reference a DataFrame column in PySpark expressions, typically for transformations or filtering.
- **when**: A conditional function that allows if-then-else logic in DataFrame transformations, similar to SQL CASE statements.
- **isnan**: A function that checks for NaN (Not a Number) values in DataFrame columns, typically used for data quality checks.
- **isnull**: A function that checks for null values in DataFrame columns, commonly used for data validation and cleaning operations.

In [1]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, isnan, isnull

StatementMeta(, 7972d5fa-4ccc-4e34-988d-a0eada4e83be, 3, Finished, Available, Finished)

## Define Parameters
- none

Note: the following is a parameter cell and will be interpreted by Pipelines as such.

## Reused Functions

## Define Fields


## Process Data

Company table

In [2]:
# Step 1: Read the data
hubspot_company_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/silver_lakehouse.Lakehouse/Tables/hubspot__company"
ins_lead_mapping_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/silver_lakehouse.Lakehouse/Tables/icrm_names_for_leads"

hubspot_company_df = spark.read.format("delta").load(hubspot_company_path)
csv_data_df = spark.read.format("delta").load(ins_lead_mapping_path)

print("Data loaded successfully")

# Step 2: Get all companies we care about
companies_df = (hubspot_company_df
    .filter(col("property_primary_unique_id").isNotNull() & col("property_icrm_account_id").isNotNull())
    .select(
        col("id").alias("company_id"),
        col("property_icrm_account_id"),
        col("property_icrm_parent_bac"),
        col("property_dealer_id_bac")
    )
    .distinct()
)

print(f"Companies: {companies_df.count()}")

# Step 3: Determine which account_id each company should use for CSV lookup
# Logic: If parent_bac is not null, find the company where dealer_id_bac = parent_bac and use its account_id
# Otherwise, use the company's own account_id

# Create a lookup: parent_bac -> account_id
parent_to_account_lookup = (companies_df
    .select(
        col("property_dealer_id_bac").alias("parent_bac"),
        col("property_icrm_account_id").alias("parent_account_id")
    )
    .filter(col("parent_bac").isNotNull())
    .dropDuplicates(["parent_bac"])  # If multiple companies have same dealer_id_bac, pick one
)

print(f"Parent lookup records: {parent_to_account_lookup.count()}")

# Step 4: Join companies with parent lookup to determine final account_id to use
# Fix: Normalize both sides by casting to long to handle .0 suffix issue
companies_with_lookup_id = (companies_df.alias("c")
    .join(parent_to_account_lookup.alias("p"), 
          col("c.property_icrm_parent_bac").cast("long") == col("p.parent_bac").cast("long"), 
          "left")
    .select(
        col("c.company_id"),
        col("c.property_icrm_account_id").alias("original_account_id"),
        col("c.property_icrm_parent_bac"),
        # Use parent's account_id if parent exists, otherwise use own account_id
        when(col("c.property_icrm_parent_bac").isNotNull(), col("p.parent_account_id"))
        .otherwise(col("c.property_icrm_account_id"))
        .alias("lookup_account_id")
    )
)

print(f"Companies with lookup logic: {companies_with_lookup_id.count()}")

# Step 5: Join with CSV data using the determined lookup_account_id
companies_with_dos_data = (companies_with_lookup_id.alias("c")
    .join(csv_data_df.alias("csv"), 
          col("c.lookup_account_id") == col("csv.uniqueaccountid"), 
          "left")
    .select(
        col("c.company_id"),
        # col("c.original_account_id"),
        # col("c.property_icrm_parent_bac"),
        # col("c.lookup_account_id"),
        
        # DOS fields with NULLIF logic
        when((col("csv.f_i_dir_of_sales") == "Unassigned") | 
             (col("csv.f_i_dir_of_sales") == "Unassigned Owner"), None)
        .otherwise(col("csv.f_i_dir_of_sales")).alias("f_i_dos_full_name"),
        
        col("csv.f_i_dir_of_sales_email").alias("f_i_dos_ally_email"),
        col("csv.f_i_dir_of_sales_hubspot_email").alias("f_i_dos_hubspot_email"),
        col("csv.f_i_dir_of_sales_mobile").alias("f_i_dos_phone_number"),
        
        when((col("csv.f_i_sr_dir_of_sales") == "Unassigned") | 
             (col("csv.f_i_sr_dir_of_sales") == "Unassigned Owner"), None)
        .otherwise(col("csv.f_i_sr_dir_of_sales")).alias("f_i_sdos_full_name"),
        
        col("csv.f_i_sr_dir_of_sales_email").alias("f_i_sdos_ally_email"),
        col("csv.f_i_sr_dir_of_sales_hubspot_email").alias("f_i_sdos_hubspot_email"),
        col("csv.f_i_sr_dir_of_sales_mobile").alias("f_i_sdos_phone_number"),
        
        when((col("csv.regional_growth_leader_1") == "Unassigned") | 
             (col("csv.regional_growth_leader_1") == "Unassigned Owner"), None)
        .otherwise(col("csv.regional_growth_leader_1")).alias("regional_growth_leader_1_full_name"),
        
        col("csv.regional_growth_leader_1_email").alias("regional_growth_leader_1_ally_email"),
        col("csv.regional_growth_leader_1_hubspot_email").alias("regional_growth_leader_1_hubspot_email"),
        col("csv.regional_growth_leader_1_mobile").alias("regional_growth_leader_1_phone_number"),
        
        when((col("csv.regional_growth_leader_2") == "Unassigned") | 
             (col("csv.regional_growth_leader_2") == "Unassigned Owner"), None)
        .otherwise(col("csv.regional_growth_leader_2")).alias("regional_growth_leader_2_full_name"),
        
        col("csv.regional_growth_leader_2_email").alias("regional_growth_leader_2_ally_email"),
        col("csv.regional_growth_leader_2_hubspot_email").alias("regional_growth_leader_2_hubspot_email"),
        col("csv.regional_growth_leader_2_mobile").alias("regional_growth_leader_2_phone_number"),
        
        when((col("csv.p_c_sr_director_of_sales") == "Unassigned") | 
             (col("csv.p_c_sr_director_of_sales") == "Unassigned Owner"), None)
        .otherwise(col("csv.p_c_sr_director_of_sales")).alias("adi_sdos_full_name"),
        
        col("csv.p_c_sr_director_of_sales_email").alias("adi_sdos_ally_email"),
        col("csv.p_c_sr_director_of_sales_hubspot_email").alias("adi_sdos_hubspot_email"),
        col("csv.p_c_sr_director_of_sales_mobile").alias("adi_sdos_phone_number"),
        
        when((col("csv.p_c_director_of_sales") == "Unassigned") | 
             (col("csv.p_c_director_of_sales") == "Unassigned Owner"), None)
        .otherwise(col("csv.p_c_director_of_sales")).alias("adi_dos_full_name"),
        
        col("csv.p_c_director_of_sales_email").alias("adi_dos_ally_email"),
        col("csv.p_c_director_of_sales_hubspot_email").alias("adi_dos_hubspot_email"),
        col("csv.p_c_director_of_sales_mobile").alias("adi_dos_phone_number"),
        
        when((col("csv.p_c_account_manager") == "Unassigned") | 
             (col("csv.p_c_account_manager") == "Unassigned Owner"), None)
        .otherwise(col("csv.p_c_account_manager")).alias("adi_account_manager_full_name"),
        
        col("csv.p_c_account_manager_email").alias("adi_account_manager_ally_email"),
        col("csv.p_c_account_manager_hubspot_email").alias("adi_account_manager_hubspot_email"),
        col("csv.p_c_account_manager_mobile").alias("adi_account_manager_phone_number")
    )
)

print(f"Final result count: {companies_with_dos_data.count()}")

# # Step 6: Test with your problem companies
# test_companies = companies_with_dos_data.filter(
#     col("company_id").isin([26521590272, 26512187672])
# ).select(
#     "company_id", 
#     "original_account_id",
#     "property_icrm_parent_bac",
#     "lookup_account_id",
#     "f_i_dir_of_sales", 
#     "f_i_dir_of_sales_email"
# )

# display("Test results for problem companies:")
# display(test_companies.collect())

# Step 7: Save to Delta table
# Show sample of the data
display("Sample of final data:")
display(companies_with_dos_data.head(10))

delta_table_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/smartsync_lakehouse.Lakehouse/Tables/hubspot_leads_notifications_for_companies"

companies_with_dos_data.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(delta_table_path)

print(f"Successfully created Delta table: hubspot_leads_notifications_for_companies")
print(f"Total records: {companies_with_dos_data.count()}")

StatementMeta(, 7972d5fa-4ccc-4e34-988d-a0eada4e83be, 4, Finished, Available, Finished)

Data loaded successfully
Companies: 47359
Parent lookup records: 46979
Companies with lookup logic: 47359
Final result count: 47359


'Sample of final data:'

SynapseWidget(Synapse.DataFrame, 94abc700-ec95-45ca-8d29-f27fd27389ac)

Successfully created Delta table: hubspot_leads_notifications_for_companies
Total records: 47359


Contact table

In [3]:
# Read the source tables - using Delta format with file paths
# DF from company table build = companies_with_dos_data
hubspot_contact_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/silver_lakehouse.Lakehouse/Tables/hubspot__contact"
hubspot_association_type_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/silver_lakehouse.Lakehouse/Tables/hubspot__association_type"
# hubspot_contact_company_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/silver_lakehouse.Lakehouse/Tables/hubspot__contact_company"
hubspot_contact_company_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/bronze_lakehouse.Lakehouse/Tables/hubspot_associations__contact_company"

hubspot_contact_df = spark.read.format("delta").load(hubspot_contact_path)
hubspot_association_type_df = spark.read.format("delta").load(hubspot_association_type_path)
hubspot_contact_company_df = spark.read.format("delta").load(hubspot_contact_company_path)

# Get distinct contacts (equivalent to contacts CTE)
contacts_df = (hubspot_contact_df
    .select(col("id").alias("contact_id"))
    .distinct()
)

print(f"Contacts count: {contacts_df.count()}")

# Filter association types (equivalent to association_type CTE)
association_type_df = (hubspot_association_type_df
    .filter((col("from_object_type") == "contact") & (col("to_object_type") == "company") & (col("id") == 1))
)

print(f"Association types count: {association_type_df.count()}")

# Get filtered associations (equivalent to associations CTE)
associations_df = (hubspot_contact_company_df.alias("a")
    .join(association_type_df.alias("b"), col("a.type_id") == col("b.id"), "inner")
    .select(col("a.*"))
)

print(f"Associations count: {associations_df.count()}")

# Create final_df by joining ins_lead_mapping_assignments with associations (equivalent to final_df CTE)
final_df = (companies_with_dos_data.alias("ins")
    .join(associations_df.alias("c"), col("ins.company_id") == col("c.company_id"), "inner")
    .select(
        col("c.contact_id"),
        col("ins.*")
    )
)

print(f"Final result count: {final_df.count()}")

# Show sample of the data
display("Sample of final data:")
display(final_df.head(10))

# Write to Delta table in smartsync_lakehouse
delta_table_path = f"abfss://Mythic@onelake.dfs.fabric.microsoft.com/smartsync_lakehouse.Lakehouse/Tables/hubspot_leads_notifications_for_contacts"

final_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(delta_table_path)

print(f"Successfully created Delta table: hubspot_leads_notifications_for_contacts")
print(f"Location: {delta_table_path}")
print(f"Total records: {final_df.count()}")

StatementMeta(, 7972d5fa-4ccc-4e34-988d-a0eada4e83be, 5, Finished, Available, Finished)

Contacts count: 257885
Association types count: 1
Associations count: 91730
Final result count: 79520


'Sample of final data:'

SynapseWidget(Synapse.DataFrame, 12ab0b2d-1821-4744-83e5-2512d768965a)

Successfully created Delta table: hubspot_leads_notifications_for_contacts
Location: abfss://Mythic@onelake.dfs.fabric.microsoft.com/smartsync_lakehouse.Lakehouse/Tables/hubspot_leads_notifications_for_contacts
Total records: 79520


DEBUGGING